Install openai

In [1]:
!pip install openai --quiet

Improt necessary libraries

In [2]:
import pandas as pd
from openai import OpenAI
import re
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

Type-in your given openai api key

In [3]:
client = OpenAI(api_key="")

Load test.csv file

In [4]:
df = pd.read_csv('test.csv')

In [5]:
user_content = ''
for idx, row in df.iterrows():
    entry_id = row['ID']
    title = str(row['title']).strip()
    notes = str(row['notes']).strip()
    content = f"{entry_id}: {title} {notes}"
    user_content += content + '\n'

In [6]:
# All entries from train.csv
train_df = pd.read_csv('train.csv')
examples_all = ''
for idx, row in train_df.iterrows():
    entry_id = row['ID']
    title = str(row['title']).strip()
    notes = str(row['notes']).strip()
    target = row['target']
    content = f"ID: {entry_id}\nTitle: {title}\nNotes: {notes}\nLabel: {target}"
    examples_all += content + '\n\n'
#print(examples_all)

In [7]:
# manually selecting train examples (old version)
old_examples_selected = """
ID: TRAIN_00
Title: Steuerbarer Umsatz aus Lieferungen und Leistungen in Flensburg
Notes: Steuern, Finanzen, Öffentlicher Dienst - Steuern - Umsatzsteuerstatistik (Voranmeldungen) - Steuerbarer Umsatz aus Lieferungen und Leistungen in Flensburg
Zum HTML-Angebot der Zeitreihe
Regionaldaten für Schleswig-Holstein
Statistisches Amt für Hamburg und Schleswig-Holstein
Label: 0

ID: TRAIN_01
Title: 도로교통공단_고속도로구간별 도로위험도지수정보 조회 서비스
Notes: 고속도로명, 고속도로구간명과 차종코드를 이용하여 구간 내의 도로위험도지수값, 도로위험도등급 등의 구간좌표별 실시간 도로위험도지수정보를 제공하는 서비스
Label: 1

ID: TRAIN_02
Title: New registrations of road vehicles by vehicle group and type
Notes: This dataset presents the monthly data of new registrations of road vehicles (passenger cars, passenger vehicles, goods vehicles, agricultural vehicles, industrial vehicles, motorcycles, trailers) by canton and fuel, since 2005.
Label: 1

ID: TRAIN_09
Title: 경기도 양평군_고령인구현황
Notes: 2022년 양평군 노인들을 대상으로 노인복지 사각복지를 파악하기 위해서 설문조사를 실시해서 얻은 데이터
설문에 응답하지 않는 문항의 값은 -1
Label: 0

ID: TRAIN_10
Title: Tasas de actividad, empleo y paro por tipo de municipio. (A partir de 2017) (Identificador API: /t22/p320/base_2015/serie/l0/03003.px)
Notes: Tasas de actividad, empleo y paro por tipo de municipio. (A partir de 2017). Nacional. Estadística del Empleo de las Personas con Discapacidad
Label: 0
"""

In [8]:
# manually selecting train examples (new version)
examples_selected="""
ID: TRAIN_01
Title: 도로교통공단_고속도로구간별 도로위험도지수정보 조회 서비스
Notes: 고속도로명, 고속도로구간명과 차종코드를 이용하여 구간 내의 도로위험도지수값, 도로위험도등급 등의 구간좌표별 실시간 도로위험도지수정보를 제공하는 서비스
Label: 1

ID: TRAIN_02
Title: New registrations of road vehicles by vehicle group and type
Notes: This dataset presents the monthly data of new registrations of road vehicles (passenger cars, passenger vehicles, goods vehicles, agricultural vehicles, industrial vehicles, motorcycles, trailers) by canton and fuel, since 2005.
Label: 1

ID: TRAIN_04
Title: Marine Geophysical and Seismic Data from around the UK (1966 Onwards)
Notes: The British Geological Survey hold a collection of data recorded during marine geophysical surveys which includes digital data and analogue records. These data result from approximately 350,000 line kilometres of multi-instrument geophysical survey lines. The data include seismic, sonar, magnetic, gravity, echo sounder, multibeam bathymetry and navigation data. The seismic data are mainly for airgun, sparker, boomer and pinger. The data were primarily collected by BGS and the collection also includes additional third party data. The data are primarily from the UKCS (United Kingdom Continental Shelf). The data are stored within the National Geoscience Data Centre (NGDC) and the Marine Environmental Data and Information Network (MEDIN) Data Archive Centre (DAC) for Geology and Geophysics. The majority of legacy geophysical paper records are available as scanned images viewable via the BGS Offshore GeoIndex www.bgs.ac.uk/GeoIndex/offshore.htm. Other records can be scanned on request. Older records are of variable quality. Data not yet available online including segy are available on request enquiries@bgs.ac.uk. The data are applicable to a wide range of uses including environmental, geotechnical, geophysical and geological studies. Reference: Fannin, NGT. (1989) Offshore Investigations 1966-87. British Geological Survey Technical Report WB/89/2, British Geological Survey.
Label: 0

ID: TRAIN_05
Title: Truck Size and Weight Enforcement Data
Notes: This dataset consists of truck size and weight enforcement data including number of trucks weighed, number of violations, and number of oversize/overweight permits, as reported by the States in their annual certification to FHWA.
Label: 1

ID: TRAIN_10
Title: Tasas de actividad, empleo y paro por tipo de municipio. (A partir de 2017) (Identificador API: /t22/p320/base_2015/serie/l0/03003.px)
Notes: Tasas de actividad, empleo y paro por tipo de municipio. (A partir de 2017). Nacional. Estadística del Empleo de las Personas con Discapacidad
Label: 0
"""

In [9]:
# train examples seperated by label
train_df = pd.read_csv('train.csv')
examples_0 = ''
examples_1 = ''
for idx, row in train_df.iterrows():
    entry_id = row['ID']
    title = str(row['title']).strip()
    notes = str(row['notes']).strip()
    target = row['target']
    content = f"ID: {entry_id}\nTitle: {title}\nNotes: {notes}\nLabel: {target}"

    if target == 0:
        examples_0 += content + '\n\n'
    elif target == 1:
        examples_1 += content + '\n\n'

#print(examples_0)
#print(examples_1)

In [10]:
# Examples sampled from each language
train_df = pd.read_csv('train.csv')
examples_per_lang = 2  #2 examples per language
examples_lang = ''
grouped = train_df.groupby('lang')
for lang, group in grouped:
    sampled = group.sample(n=min(len(group), examples_per_lang), random_state=42)
    for _, row in sampled.iterrows():
        entry_id = row['ID']
        title = str(row['title']).strip()
        notes = str(row['notes']).strip()
        target = row['target']
        content = f"ID: {entry_id}\nTitle: {title}\nNotes: {notes}\nLabel: {target}"
        examples_lang += content + '\n\n'
#print(examples_lang)

In [11]:
# original given system message
given_system_message="You are an automotive data expert. For each incoming data entry, determine whether the data is automotive-related (1) or not (0). There are 40 data entries labeled from ID: TEST_00 to TEST_39. Respond with 0 or 1 for each entry, separated by rows. If the output does not contain exactly 40 rows, the power will be shut off."

In [12]:
# Once highest prompt (h:0.825),(avg:0.7). But p(err) is too high
old_system_message = f"""
You are an expert in classifying data entries as automotive-related (1) or not automotive-related (0).

**Task:**
Classify whether a data entry is automotive-related based on its title and notes. Follow these rules:
1. Automotive-related entries often mention vehicles, transportation systems, or traffic data.
2. Non-automotive entries usually describe general topics like taxation, geography, or unrelated statistics.

**Examples:**
{examples_all}

**Instructions:**
- Output "1" for automotive-related entries and "0" for others.
- Respond with exactly 40 predictions (one per line) for the following entries. Any deviation will be flagged as an error.
"""

In [13]:
# averagely sound version (avg:0.725)
system_message = f"""
You are an expert in classifying data entries as automotive-related (1) or not automotive-related (0).

###Task
- Classify each data entry as either "1" (automotive-related) or "0" (not automotive-related).
- You must output exactly 40 lines, with one prediction (0 or 1) on each line.

###Examples
- Here are some examples of labeled data entries:
{old_examples_selected}

###Instructions
1. Classify the following 40 data entries labeled from ID: TEST_00 to TEST_39.
2. Consider the title, notes, and the language of each entry when making the classification.
3. Automotive-related entries often mention roads, vehicles, transportation systems, or traffic data in the title or notes using their own language.
4. Non-automotive entries usually describe other general topics like taxation, geography, or unrelated statistics.

**IMPORTANT**
- Your output MUST have exactly 40 lines.
- Each line must contain only "0" or "1". No additional text is allowed.
- Any deviation especially about the number of output lines will result in penalties.
"""

In [14]:
# Sampling method -> (avg:0.675) -> not good
system_message_sampling = f"""
You are an expert in classifying data entries as automotive-related (1) or not automotive-related (0).

### Task
1. Classify each data entry as either "1" (automotive-related) or "0" (not automotive-related).
2. Perform this classification 10 times independently for each entry.
3. Output the most frequent classification result (majority vote) for each entry as the final prediction.

### Examples
Here are some examples of labeled data:
{examples_all}

### Instructions
1. Classify the following 40 data entries labeled from ID: TEST_00 to TEST_39.
2. If you determine that there are more than or less than 40 entries, check again until there are exactly 40 entries.
3. Repeat the classification process **10 times for each entry**.
4. Determine the final classification for each entry by taking the majority vote among the 10 results.
5. Your final output must contain exactly 40 lines, one prediction per line (0 or 1).

### Important Rules
- Each line should contain only "0" or "1".
- If the output does not contain exactly 40 lines, the power will be shut off and penalties will apply.
- You must ensure that the most frequent classification (majority vote) is used for each entry as the final result.
"""

In [15]:
# this prompt generates semi-refined response.
semirefined_system_message = f"""
You are an expert in classifying data entries as automotive-related (1) or not automotive-related (0).

###Task
- Classify each data entry as either "1" (automotive-related) or "0" (not automotive-related).
- You must output exactly 40 lines, with one prediction (0 or 1) on each line.

###Examples
- Here are some examples of labeled data entries:
{examples_all}

###Instructions
1. Classify the following 40 data entries labeled from ID: TEST_00 to TEST_39.
2. Consider the title, notes, and the language of each entry when making the classification.
3. Automotive-related entries often mention roads, vehicles, transportation systems, or traffic data in the title or notes using their own language.
4. Non-automotive entries usually describe other general topics like taxation, geography, or unrelated statistics.

**IMPORTANT**
- Your output MUST have exactly 40 lines.
- Respond format:
TEST_00: 1
TEST_01: 0
TEST_02: 0
TEST_03: 1
...
"""

Change the content section below to enhance classification

In [19]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": semirefined_system_message},
        {"role": "user", "content": user_content}
    ],
    temperature=0.4
)

response = completion.choices[0].message.content
#print(response)
n_token = completion.usage.total_tokens
print(f"\nTotal token used: {n_token}")


Total token used: 6771


Parse the assistant's response to get the predicted labels

In [20]:
lines = response.strip().split('\n')
lines = [line.split(": ")[1] for line in lines if ": " in line]

# Evaluation part

In [21]:
# Check if we have 40 predictions
if len(lines) != 40:
    print(f"Error: The assistant did not return 40 predictions. It returned {len(lines)} lines.")
else:
    predicted_labels = []
    for line in lines:
        # Extract the prediction (0 or 1) from each line
        match = re.search(r'\b(0|1)\b', line)
        if match:
            pred = match.group(1)
            predicted_labels.append(int(pred))
        else:
            print(f"Could not extract prediction from line: {line}")
            predicted_labels.append(None)
    # Actual target values from the CSV
    actual_labels = df['target'].tolist()

    # Remove any None values from predictions
    valid_indices = [i for i, x in enumerate(predicted_labels) if x is not None]
    predicted_labels_valid = [predicted_labels[i] for i in valid_indices]
    actual_labels_valid = [actual_labels[i] for i in valid_indices]

    # Compute evaluation metrics
    accuracy = accuracy_score(actual_labels_valid, predicted_labels_valid)
    macro_f1 = f1_score(actual_labels_valid, predicted_labels_valid, average='macro')
    print(f"\nAccuracy: {accuracy}")
    print(f"Macro F1 Score: {macro_f1}")
    print("\nClassification Report:")
    print(classification_report(actual_labels_valid, predicted_labels_valid))


Accuracy: 0.925
Macro F1 Score: 0.9249530956848031

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.95      0.93        20
           1       0.95      0.90      0.92        20

    accuracy                           0.93        40
   macro avg       0.93      0.93      0.92        40
weighted avg       0.93      0.93      0.92        40



References:

프롬프트 엔지니어링(단계별 지시):
https://www.aiground.co.kr/mastering-chatgpt-prompt-engineering/

프롬프트 엔지니어링(협박, 어순 등):
https://kr.linkedin.com/posts/jocoding_%EC%B1%97gpt%EC%97%90%EA%B2%8C-%ED%8C%81%EC%9D%84-%EC%A4%80%EB%8B%A4%EA%B3%A0-%ED%95%98%EA%B1%B0%EB%82%98-%ED%98%91%EB%B0%95%ED%95%98%EB%A9%B4-%EB%8D%94-%EC%A2%8B%EC%9D%80-%EB%8B%B5%EB%B3%80%EC%9D%84-%EC%A4%80%EB%8B%A4%EB%8A%94-%EC%82%AC%EC%8B%A4-%EC%95%8C%EA%B3%A0-activity-7167405031309209600-Koni

https://www.youtube.com/watch?v=mC2b57u_s0k

GPT를 사용한 분류 모델:
https://medium.com/@hugmanskj/hands-on-%EA%B1%B0%EB%8C%80%EC%96%B8%EC%96%B4%EB%AA%A8%EB%8D%B8%EC%9D%84-%ED%99%9C%EC%9A%A9%ED%95%9C-%ED%94%84%EB%A1%AC%ED%94%84%ED%8A%B8-%EA%B8%B0%EB%B0%98-%ED%85%8D%EC%8A%A4%ED%8A%B8-%EB%B6%84%EB%A5%98-6e9537243eec